In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from smart_open import smart_open
# nltk.download('stopwords')  # run once
from nltk.corpus import stopwords
# stop_words = stopwords.words('english')
from gensim import corpora
from pprint import pprint
# from jupyterthemes import jtplot
# jtplot.style(theme='monokai', context='notebook',
#              ticks=True, grid=False)
import math
from sklearn import datasets
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt

import operator

# jtplot.style(theme='monokai', context='notebook',
#              ticks=True, grid=False)

In [2]:
df= pd.read_csv("final_clustering_df.csv")
X = np.asarray(df.iloc[:,[1,2]])

In [3]:
import pickle
with open('cluster_result_median', 'rb') as f:
     index1 = pickle.load(f)

index1

array([array([    3,     6,     8, ..., 20231, 20234, 20240], dtype=int64),
       array([    2,    22,    31, ..., 20232, 20233, 20237], dtype=int64),
       array([    1,     7,    10, ..., 20238, 20239, 20241], dtype=int64),
       array([    0,     4,     5, ..., 20221, 20222, 20226], dtype=int64)],
      dtype=object)

In [8]:
def euclidean_distance_word(dat, rand_medians,word_sim):
    dist_list = []
    for i in range(len(rand_medians)):
        summation = np.sum((X-rand_medians[i])**2, axis=1)
        summation = normalize(summation.reshape(-1,1))
        summation = summation + (1-word_sim).reshape(-1,1)
#         summation = word_sim
        dist_list.append(np.sqrt(summation))
    dist_list = np.asarray(dist_list)
    return (dist_list)

def word_sim_calc(df,item):
    df = df
    file_docs = []
    for i in range(len(df)):
        tokens = sent_tokenize(df['Names'][i])
        file_docs.append(tokens)
    mydict = corpora.Dictionary([simple_preprocess((" ".join(line))) for line in file_docs])
    corpus = [mydict.doc2bow(simple_preprocess((" ".join(line)))) for line in file_docs]
    tfidf = gensim.models.TfidfModel(corpus)
    workdir = r'C:\Users\akhil\OneDrive - University of Oklahoma\Subjects\Data Mining\Project\ '
    sims = gensim.similarities.Similarity(workdir,tfidf[corpus],num_features=len(mydict))
    query_doc_tf_idf = tfidf[corpus[item]]
    word_sim = sims[query_doc_tf_idf]
    return (word_sim)\

def similar_prod(item,no_item,data):
    user = []
    X = np.asarray(data.iloc[:,[1,2]])
    user.append(X[item])
    wordsim = word_sim_calc(data,item)
    dist_array = euclidean_distance_word(X,user,wordsim)
    dict = { i : dist_array[:,i] for i in range(0, dist_array.shape[1] ) }
    sorted_dict = sorted(dict.items(), key=operator.itemgetter(1))
    similar_prod_index = sorted_dict[0:(no_item+1)]
    result = df.iloc[np.asarray(similar_prod_index)[:,0]]
    name = result['Names'].tolist()
    price = result['Price'].tolist()
    rating = result['Rating'].tolist()
    return name,price,rating

In [13]:
ind = int(input("Select product index : ") )
num = int(input("Select no of similar products : ") )
similar_prod(ind,num,df)

# for i in range (4):
#     if ((np.asarray(np.where(index1[i]==ind)).size) >0):
#         print ("The product is found in cluster number:", (i+1) )
#         print("The products in this cluster have the following price and ratings structures\n", 
#               df.iloc[index1[i]].describe().loc[['mean','min','max']])
#     df2 = df.iloc[index1[i]]

Select product index : 16699
Select no of similar products : 3


(['Dungeons &amp; Dragons Starter Kit: 1',
  'SmartDealsPro 8 x 7-Die Series 8 Colors Sparkle Dungeons and Dragons DND RPG MTG Table Games Dice',
  "Magic The Gathering Origins Deck Builder's Tool Kit",
  'WHATWEARS Sd Die D4 D6 D8 D10 D12 D20 MTG RPG D&amp;D DND Poly Dices Pearl Purple 7pcs'],
 [25.34, 25.34, 25.34, 25.34],
 [4.7, 4.7, 4.7, 4.7])

In [55]:
def clustering_res_median(ind):
    for i in range (4):
        if ((np.asarray(np.where(index1[i]==ind)).size) >0):
            clust_num = (i+1)
#             print ("The product is found in cluster number:",clust_num  )
            pmean = df["Price"].iloc[index1[i]].describe().loc[['mean']].values
            pmax = df["Price"].iloc[index1[i]].describe().loc[['max']].astype(int).values
            pmin = df["Price"].iloc[index1[i]].describe().loc[['min']].astype(int).values
            rmean = df["Rating"].iloc[index1[i]].describe().loc[['mean']].astype(int).values
            rmax = df["Rating"].iloc[index1[i]].describe().loc[['max']].astype(int).values
            rmin = df["Rating"].iloc[index1[i]].describe().loc[['min']].astype(int).values
            return (pmean,pmax,pmin,rmean,rmax,rmin)

In [26]:
pmean

NameError: name 'pmean' is not defined

In [36]:
df["Rating"].iloc[index1[2]].describe().loc[['mean']]

mean    4.227291
Name: Rating, dtype: float64

In [21]:
att_list = ["Price","Rating"]
var_list = ['pmean','pmax','pmin','rmean','rmax','rmin']